### Simple Neural network based on demographic data for cardiovascular death prediction

In [32]:
import tensorflow as tf
import keras 
import pandas as pd

#### Load data and clean data from demographics file

In [60]:
demographics = pd.read_csv('Demographics.csv')
demographics.head()

,Unnamed: 0,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,...,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,ATHERO_DIAGNOSIS_FLAG,HEART_DEATH_FLAG,CAUSE
0,0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,UKNOWN,...,Medicare,TRANSFER FROM HOSP/EXTRAM,1,0,1,0,0,0,0,STATUS EPILEPTICUS
1,1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,UKNOWN,UKNOWN,...,Medicare,EMERGENCY ROOM ADMIT,1,0,1,1,0,0,0,HEAD BLEED
2,2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,UKNOWN,...,Private,CLINIC REFERRAL/PREMATURE,2,0,1,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA
3,3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,UKNOWN,...,Medicare,EMERGENCY ROOM ADMIT,2,0,1,0,0,0,0,SUBARACHNOID HEMORRHAGE
4,4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,UKNOWN,...,Private,EMERGENCY ROOM ADMIT,2,0,1,0,0,0,0,"GLIOBLASTOMA,NAUSEA"


In [61]:
# First, define atherosclerosis diagnoses from non-atherosclerosis diagnoses
athero_pre = demographics[demographics['OLD_FLAG']==0]
athero_pos = athero_pre[athero_pre['ATHERO_DIAGNOSIS_FLAG']== 1]
athero_neg = athero_pre[athero_pre['ATHERO_DIAGNOSIS_FLAG']==0]

# Clean data sets
del athero_neg['CAUSE']
del athero_pos['CAUSE']

del athero_neg['ATHERO_DIAGNOSIS_FLAG']
del athero_pos['ATHERO_DIAGNOSIS_FLAG']

del athero_neg['OLD_FLAG']
del athero_pos['OLD_FLAG']

del athero_neg['OUTSIDE_DEATH_FLAG']
del athero_pos['OUTSIDE_DEATH_FLAG']

del athero_neg['SUBJECT_ID']
del athero_pos['SUBJECT_ID']

del athero_neg['DOB']
del athero_pos['DOB']

del athero_neg['DOD']
del athero_pos['DOD']

In [62]:
athero_pos['DOA']
del athero_pos['DOA']
del athero_neg['DOA']

athero_neg['HEART_ATTACK_FLAG']
del athero_neg['HEART_ATTACK_FLAG']
del athero_pos['HEART_ATTACK_FLAG']

del athero_pos['Unnamed: 0']
athero_pos.head()

,GENDER,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,DEATH_FLAG,HEART_DEATH_FLAG
9,M,69.641096,WHITE,MARRIED,UKNOWN,CATHOLIC,Private,TRANSFER FROM HOSP/EXTRAM,4,1,0
12,F,69.005479,WHITE,MARRIED,ENGL,PROTESTANT QUAKER,Medicare,EMERGENCY ROOM ADMIT,2,1,0
17,M,87.882192,WHITE,MARRIED,UKNOWN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,2,1,0
19,F,76.871233,WHITE,MARRIED,PORT,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,4,1,1
22,F,85.726027,BLACK/AFRICAN AMERICAN,WIDOWED,UKNOWN,CATHOLIC,Medicare,EMERGENCY ROOM ADMIT,2,1,0


In [63]:
# Create Outcome data sets
athero_heartdeath = pd.Series(athero_pos['HEART_DEATH_FLAG'])
athero_death = pd.Series(athero_pos['DEATH_FLAG'])

In [64]:
del athero_pos['HEART_DEATH_FLAG']
del athero_pos['DEATH_FLAG']

In [65]:
# Get dummies
athero_pos = pd.get_dummies(athero_pos, columns=['GENDER','ETHNICITY','MARITAL_STATUS', 'LANGUAGE', 'RELIGION', 'INSURANCE', 'ADMISSION_LOCATION'])

In [66]:
# Check outcome numbers
print(athero_heartdeath.value_counts())
print(athero_death.value_counts())

0    10001
1      202
Name: HEART_DEATH_FLAG, dtype: int64
0    6576
1    3627
Name: DEATH_FLAG, dtype: int64


In [67]:
# Normalize data
from sklearn import preprocessing
athero_pos = preprocessing.scale(athero_pos)

In [68]:
# Test/ train 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(athero_pos, athero_death, test_size=0.20, random_state=42)

#### Create simple neural network as a baseline model

In [69]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(0)

In [70]:
# Create model
model = Sequential()
model.add(Dense(80, input_dim=121 , activation = 'relu'))
model.add(Dense(60, activation = 'relu'))
model.add(Dense(40, activation = 'relu'))
model.add(Dense(20, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [71]:
# Compile model
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [72]:
# checkpoint
from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [73]:
model.fit(X_train, y_train, batch_size=20, nb_epoch=150, verbose=1, callbacks=callbacks_list, validation_data=(X_test, y_test), shuffle=True)

/Users/jimmy/anaconda3/envs/Tensorflow/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 8162 samples, validate on 2041 samples
Epoch 1/150
8162/8162 [==============================] - 1s 125us/step - loss: 0.5769 - acc: 0.7031 - val_loss: 0.5374 - val_acc: 0.7364

Epoch 00001: val_acc improved from -inf to 0.73640, saving model to weights.best.hdf5
Epoch 2/150
8162/8162 [==============================] - 1s 85us/step - loss: 0.5295 - acc: 0.7362 - val_loss: 0.5353 - val_acc: 0.7398

Epoch 00002: val_acc improved from 0.73640 to 0.73983, saving model to weights.best.hdf5
Epoch 3/150
8162/8162 [==============================] - 1s 81us/step - loss: 0.5155 - acc: 0.7488 - val_loss: 0.5490 - val_acc: 0.7310

Epoch 00003: val_acc did not improve
Epoch 4/150
8162/8162 [==============================] - 1s 81us/step - loss: 0.5099 - acc: 0.7517 - val_loss: 0.5363 - val_acc: 0.7447

Epoch 00004: val_acc improved from 0.73983 to 0.74473, saving model to weights.best.hdf5
Epoch 5/150
8162/8162 [==============================] - 1s 83us/step - loss: 0.5017 - acc: 0.7546 - v


Epoch 00046: val_acc did not improve
Epoch 47/150
8162/8162 [==============================] - 1s 87us/step - loss: 0.3055 - acc: 0.8540 - val_loss: 1.1067 - val_acc: 0.7036

Epoch 00047: val_acc did not improve
Epoch 48/150
8162/8162 [==============================] - 1s 87us/step - loss: 0.3215 - acc: 0.8478 - val_loss: 1.0187 - val_acc: 0.7036

Epoch 00048: val_acc did not improve
Epoch 49/150
8162/8162 [==============================] - 1s 80us/step - loss: 0.3151 - acc: 0.8502 - val_loss: 1.0717 - val_acc: 0.7026

Epoch 00049: val_acc did not improve
Epoch 50/150
8162/8162 [==============================] - 1s 87us/step - loss: 0.3164 - acc: 0.8542 - val_loss: 1.0011 - val_acc: 0.6923

Epoch 00050: val_acc did not improve
Epoch 51/150
8162/8162 [==============================] - 1s 85us/step - loss: 0.3031 - acc: 0.8580 - val_loss: 1.0531 - val_acc: 0.6948

Epoch 00051: val_acc did not improve
Epoch 52/150
8162/8162 [==============================] - 1s 82us/step - loss: 0.3011 -


Epoch 00093: val_acc did not improve
Epoch 94/150
8162/8162 [==============================] - 1s 92us/step - loss: 0.2635 - acc: 0.8734 - val_loss: 1.3640 - val_acc: 0.6840

Epoch 00094: val_acc did not improve
Epoch 95/150
8162/8162 [==============================] - 1s 92us/step - loss: 0.2556 - acc: 0.8787 - val_loss: 1.4000 - val_acc: 0.6903

Epoch 00095: val_acc did not improve
Epoch 96/150
8162/8162 [==============================] - 1s 86us/step - loss: 0.2630 - acc: 0.8790 - val_loss: 1.3762 - val_acc: 0.6992

Epoch 00096: val_acc did not improve
Epoch 97/150
8162/8162 [==============================] - 1s 86us/step - loss: 0.2558 - acc: 0.8763 - val_loss: 1.4405 - val_acc: 0.7016

Epoch 00097: val_acc did not improve
Epoch 98/150
8162/8162 [==============================] - 1s 84us/step - loss: 0.2553 - acc: 0.8805 - val_loss: 1.3749 - val_acc: 0.6948

Epoch 00098: val_acc did not improve
Epoch 99/150
8162/8162 [==============================] - 1s 86us/step - loss: 0.2701 -

8162/8162 [==============================] - 1s 80us/step - loss: 0.2325 - acc: 0.8852 - val_loss: 1.6379 - val_acc: 0.6952

Epoch 00140: val_acc did not improve
Epoch 141/150
8162/8162 [==============================] - 1s 80us/step - loss: 0.2449 - acc: 0.8868 - val_loss: 1.6677 - val_acc: 0.6874

Epoch 00141: val_acc did not improve
Epoch 142/150
8162/8162 [==============================] - 1s 85us/step - loss: 0.2446 - acc: 0.8823 - val_loss: 1.5542 - val_acc: 0.6830

Epoch 00142: val_acc did not improve
Epoch 143/150
8162/8162 [==============================] - 1s 91us/step - loss: 0.2318 - acc: 0.8878 - val_loss: 1.5618 - val_acc: 0.6928

Epoch 00143: val_acc did not improve
Epoch 144/150
8162/8162 [==============================] - 1s 92us/step - loss: 0.2256 - acc: 0.8917 - val_loss: 1.6059 - val_acc: 0.6850

Epoch 00144: val_acc did not improve
Epoch 145/150
8162/8162 [==============================] - 1s 91us/step - loss: 0.2291 - acc: 0.8899 - val_loss: 1.6069 - val_acc: 0.6

In [74]:
# Load model 
model.load_weights("weights.best.hdf5")

# estimate accuracy on test data set using loaded weights
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 74.47%


#### So with a simple neural network focused on demographic data only we have an accuracy of 74%, a 5% improvement over XGBoost